In [1]:
# %load startup
import numpy as np
from pathlib import Path
import time

from IPython.core.display import display, HTML
display(HTML("<style>"
             + "#notebook { padding-top:0px !important; } "
             + ".container { width:100% !important; } "
             + ".end_space { min-height:0px !important; } "
             + "</style>"))

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

import pandas as pd
pd.options.display.max_colwidth = 100
pd.options.display.precision = 3

import pprint
pp = pprint.PrettyPrinter(indent=4, width=100)

import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(module)s:%(lineno)d %(funcName)s %(levelname)s %(message)s')

In [2]:
from model_config import ModelConfig

def show_result(game_result_path, verbose=False):
    game_result_path = Path(game_result_path)
    paths = [(int(f.stem.split('_')[1]), f) for f in (game_result_path.glob('**/*.csv'))]
    paths.sort(key=lambda x: x[0])
    for game_number, csv_file in paths:
        game_result_path = csv_file.parent
        game_result_pd = pd.read_csv(csv_file)
        if len(game_result_pd) < 10:
            continue
        if 'reward' in game_result_pd:
            rewards = game_result_pd['reward'].values
            win_rate = np.sum(rewards>0)/len(rewards) 
        else:
            assert 'black_win' in game_result_pd
            win_rate = float(np.mean(game_result_pd['black_win'].values))
        median_time = np.median(game_result_pd['time'].values)
        game_config_file = game_result_path/f'config_{game_number}.yaml'
        if game_config_file.exists():
            config = ModelConfig.from_yaml(game_config_file)
            print(f'{str(csv_file)}\n B={win_rate}, '
                  f'{len(game_result_pd)}/{median_time:.1f}, '
                  f'MC={config.mcts_num_rollout}/{config.mcts_c_puct}/{config.mcts_dirichlet_alpha}/{config.mcts_tao_threshold}, '
                  f'K={config.komi}, '
                  f'{config.weight_root}')
        else:
            print(f'{str(csv_file)}\n B={win_rate}, '
                  f'#={len(game_result_pd)}, '
                  f'T={median_time:.1f}')
        if verbose:
            print(game_result_pd.describe(percentiles=[0.5]))

In [6]:
show_result('/home/peng_liu/src/tmp/game_result/mcts_vs_pachi/', verbose=False)

0: B=0.0, #=100, T=306.842
1: B=0.156, #=1000, T=35.106, S=1000, K=0.0, W=
2: B=0.16, #=1000, T=39.196, S=1000, K=0.0, W=
4: B=0.01, #=100, T=11.507, S=100, K=0.5, W=
5: B=0.0, #=10, T=42.172, S=500, K=0.5, W=
6: B=0.031, #=1000, T=75.218, S=1000, K=0.5, W=/tmp/test_nn_guided_mcts/4/
7: B=0.173, #=1000, T=83.536, S=1000, K=0.0, W=
8: B=0.0, #=100, T=86.988, S=1000, K=0.5, W=


In [3]:
show_result('/home/peng_liu/src/tmp/game_result/nn_guided_mcts_vs_pachi/', verbose=True)

/home/peng_liu/src/tmp/game_result/nn_guided_mcts_vs_pachi/game_1.csv
 B=0.0, 10/151.5, MC=500/0.2/0.0/20, K=0.5, /tmp/test_nn_guided_mcts/1/
       Unnamed: 0     plys  reward   score     time
count      10.000   10.000    10.0  10.000   10.000
mean        4.500   95.800    -1.0  62.000  126.545
std         3.028   43.893     0.0  32.531   57.513
min         0.000    6.000    -1.0   0.500    8.439
50%         4.500  114.000    -1.0  75.500  151.465
max         9.000  124.000    -1.0  81.500  160.492
/home/peng_liu/src/tmp/game_result/nn_guided_mcts_vs_pachi/game_2.csv
 B=0.0, 10/138.9, MC=500/0.2/0.0/20, K=0.5, /tmp/test_nn_guided_mcts/2/
       Unnamed: 0     plys  reward   score     time
count      10.000   10.000    10.0  10.000   10.000
mean        4.500   75.800    -1.0  44.800  102.252
std         3.028   48.939     0.0  38.416   65.629
min         0.000   14.000    -1.0   0.500   19.382
50%         4.500  104.000    -1.0  61.500  138.947
max         9.000  126.000    -1.0  81.5

/home/peng_liu/src/tmp/game_result/nn_guided_mcts_vs_pachi/game_19.csv
 B=0.0, 10/78.1, MC=500/0.2/0.0/7, K=0.5, /tmp/test_nn_guided_mcts/5/
       Unnamed: 0  black_win  max_search_depth  max_visit_ratio  min_search_depth  min_visit_ratio     plys  score     time
count      10.000       10.0            10.000             10.0              10.0             10.0   10.000   10.0   10.000
mean        4.500        0.0            12.900              0.0               1.0              1.0   76.800   29.6   80.745
std         3.028        0.0             1.853              0.0               0.0              0.0   19.938   24.7   19.630
min         0.000        0.0            12.000              0.0               1.0              1.0   56.000    7.5   57.472
50%         4.500        0.0            12.000              0.0               1.0              1.0   70.000   23.0   78.074
max         9.000        0.0            18.000              0.0               1.0              1.0  112.000   78.5 

In [26]:
df = pd.read_csv('/home/peng_liu/src/tmp/game_result/nn_guided_mcts_vs_pachi/game_24.csv')
print(df)

   Unnamed: 0  black_win  max_search_depth  max_visit_ratio  min_visit_ratio  plys  score     time
0           0          0                27              1.0            0.059   136   81.5  460.217
1           1          0                26              1.0            0.042   122   75.5  432.714
2           2          0                35              1.0            0.098   118   81.5  335.451
3           3          0                30              1.0            0.032   124   78.5  423.986
4           4          0                25              1.0            0.060   120   78.5  410.471
5           5          0                27              1.0            0.048   102   70.5  348.276
6           6          0                24              1.0            0.091    86   33.5  293.879
7           7          0                28              1.0            0.098   116   78.5  394.381
8           8          0                27              1.0            0.027    98   45.5  330.965
9         